Урок 2. CNN. Детектирование
---

**Домашнее задание**: Используя файл Web2.ipynb (он в полезных ссылках):

1. Вам нужно будет загрузить и оформить датасет котиков, собак https://www.kaggle.com/andrewmvd/dog-and-cat-detection
2. *Нужно будет произвести аугментацию данных
3. Затем обучить нейронную сеть на получившемся датасете